> Added data augmentation (flip, rotation, zoom, contrast).

> Added .cache() + .prefetch() for speed.

> Increased embedding_dim → 256.

> Changed margin → 0.5.

> Fine-tuning last 20 ResNet50 layers.

> Switched to AdamW + CosineDecayRestarts LR schedule.

> Added ReduceLROnPlateau for stability.

In [ ]:
! pip install datasets pillow tqdm

In [ ]:
import os
import os
import random
from datasets import load_dataset
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm


DATA_DIR = "/content/polyvore_images"
os.makedirs(DATA_DIR,exist_ok=True)

dataset = load_dataset("Marqo/polyvore",split="data")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data-00000-of-00006.parquet:   0%|          | 0.00/428M [00:00<?, ?B/s]

data-00001-of-00006.parquet:   0%|          | 0.00/421M [00:00<?, ?B/s]

data-00002-of-00006.parquet:   0%|          | 0.00/416M [00:00<?, ?B/s]

data-00003-of-00006.parquet:   0%|          | 0.00/416M [00:00<?, ?B/s]

data-00004-of-00006.parquet:   0%|          | 0.00/422M [00:00<?, ?B/s]

data-00005-of-00006.parquet:   0%|          | 0.00/409M [00:00<?, ?B/s]

Generating data split:   0%|          | 0/94096 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

# Function to save and resize images from bytes
def save_and_resize(image_bytes, save_path):
    if os.path.exists(save_path):
        return save_path
    try:
        img = Image.open(BytesIO(image_bytes)).convert("RGB")
        img = img.resize((224, 224))
        img.save(save_path, format="JPEG", quality=85)
        return save_path
    except Exception as e:
        print("Error saving image:", e)
        return None

local_paths = []

for row in tqdm(dataset):
    set_id = row["item_ID"]
    category = row["category"]
    img = row["image"]   # already a PIL Image object (JpegImageFile)

    # Ensure correct format & resize
    img = img.convert("RGB")
    img = img.resize((224, 224))

    fname = f"{set_id}.jpg"
    f_path = os.path.join(DATA_DIR, fname)
    img.save(f_path, format="JPEG", quality=85)

    local_paths.append({
        "set_id": set_id,
        "category": category,
        "local_path": f_path
    })

# Convert to DataFrame
df = pd.DataFrame(local_paths)
df.to_parquet("polyvore_local.parquet")
print(f"Saved {len(df)} images and metadata")

100%|██████████| 94096/94096 [05:46<00:00, 271.49it/s]


Saved 94096 images and metadata


In [ ]:
categories_to_remove=[
    "Accent Chairs",
    "Accent Tables",
    "Aprons",
    "Armoires",
    "Bakeware",
    "Bar Cabinets",
    "Bar Tools",
    "Barstools",
    "Bath",
    "Bath & Body",
    "Bath Accessories",
    "Bath Rugs & Mats",
    "Bath Towels",
    "Beach Towels",
    "Beauty Accessories",
    "Beauty Products",
    "Bed Accessories",
    "Bed Pillows",
    "Bedding",
    "Beds",
    "Bedspreads",
    "Benches",
    "Blankets",
    "Blow Dryers & Irons",
    "Blush",
    "Body Art",
    "Body Cleansers",
    "Body Moisturizers",
    "Bookcases",
    "Books",
    "Brushes & Combs",
    "Cabinets",
    "Candles & Candleholders",
    "Ceiling Lights",
    "Chairs",
    "Cheek Bronzer",
    "Cheek Makeup",
    "Children's Bedding",
    "Children's Decor",
    "Children's Furniture",
    "Children's Room",
    "Cleaning",
    "Clocks",
    "Comforters",
    "Concealer",
    "Cookbooks",
    "Cookware",
    "Costumes",
    "Curtain Rods",
    "Curtains",
    "Cutlery",
    "Decorative Hardware",
    "Deodorant",
    "Desk Lamps",
    "Dining Chairs",
    "Dining Tables",
    "Dinnerware",
    "Dressers",
    "Drinkware",
    "Duvet Covers",
    "Electronics",
    "Entertainment Units",
    "Eye Care",
    "Eye Makeup",
    "Eyelash Curlers",
    "Eyeliner",
    "Eyeshadow",
    "Eyewear",
    "Face Care",
    "Face Cleansers",
    "Face Makeup",
    "Face Masks",
    "Face Moisturizers",
    "Face Toners",
    "False Eyelashes",
    "Fans",
    "File Cabinets",
    "Fireplace Accessories",
    "Flatware",
    "Floor Lamps",
    "Flooring",
    "Floral Decor",
    "Font",
    "Food & Drink",
    "Food Storage Containers",
    "Foundation",
    "Fragrance",
    "Frames",
    "Furniture",
    "Garden Tools",
    "Gift Cards",
    "Gift Sets & Kits",
    "Hair Color",
    "Hair Conditioner",
    "Hair Removal",
    "Hair Shampoo",
    "Hair Styling Tools",
    "Haircare",
    "Hammocks & Swings",
    "Holiday Decorations",
    "Home Decor",
    "Home Fragrance",
    "Home Improvement",
    "Hosiery",
    "Jewelry Storage",
    "Kitchen & Dining",
    "Kitchen Gadgets & Tools",
    "Kitchen Linens",
    "Lighting",
    "Lip Gloss",
    "Lip Makeup",
    "Lip Pencils",
    "Lip Stain",
    "Lip Treatments",
    "Lipstick",
    "Loafers & Moccasins",
    "Makeup",
    "Makeup Brushes",
    "Makeup Primer",
    "Makeup Remover",
    "Makeup Tools",
    "Manicure Tools",
    "Mascara",
    "Media",
    "Men's Deodorant",
    "Men's Fragrance",
    "Men's Gift Sets & Kits",
    "Men's Grooming",
    "Men's Grooming Bags",
    "Men's Hair Care",
    "Men's Shaving",
    "Men's Skincare",
    "Men's Tech Accessories",
    "Men's Umbrellas",
    "Mirrors",
    "Nail Care",
    "Nail Polish",
    "Nail Treatments",
    "Napkin Rings",
    "Nightstands",
    "Nursery Furniture",
    "Office Accessories",
    "Office Chairs",
    "Oral Care",
    "Ottomans",
    "Outdoor Benches",
    "Outdoor Chairs",
    "Outdoor Decor",
    "Outdoor Lighting",
    "Outdoor Loungers & Day Beds",
    "Outdoor Patio Sets",
    "Outdoor Stools",
    "Outdoor Tables",
    "Outdoors",
    "Paint",
    "Panel Screens",
    "Party Supplies",
    "Patio Furniture",
    "Patio Umbrellas",
    "Quilts & Coverlets",
    "Rugs",
    "Serveware",
    "Sharpeners",
    "Sheets & Pillowcases",
    "Shoulder Bags",
    "Shower Curtains",
    "Sideboards",
    "Skincare",
    "Sleepwear",
    "Small Appliances",
    "Small Storage",
    "Sofas",
    "Standing Fans",
    "Stationery",
    "Stools",
    "Storage & Organization",
    "Storage & Shelves",
    "Styling Products",
    "Sun Care",
    "Table Lamps",
    "Table Linens",
    "Tables",
    "Tech Accessories",
    "Throw Pillows",
    "Tinted Moisturizer",
    "Toys",
    "Tweezers & Brow Tools",
    "Umbrellas",
    "Vases",
    "Wall Lights",
    "Wallpaper",
    "Window Blinds",
    "Window Treatments",
]

In [ ]:
clean_df= df[~df['category'].isin(categories_to_remove)].reset_index(drop=True)

print(f"length of the cleaned dataset {len(clean_df)}")

clean_df.to_parquet('cleaned_parquet.parquet')


length of the cleaned dataset 73991


In [ ]:
import itertools
import random
import pandas as pd

def make_triplets(items_df,max_triplets_per_of=50,seed=24):
  rng = random.Random(seed)

  items_df= items_df.copy()
  items_df['outfit_id']= items_df['set_id'].str.split('_').str[0]

  grouped = items_df.groupby('outfit_id')['local_path'].apply(list).to_dict()
  outfit_ids= list(grouped.keys())

  triplets=[]

  for oid, items in grouped.items():
    if len(items) < 2:

      continue

    ap_pairs = list(itertools.combinations(items,2))

    if len(ap_pairs)> max_triplets_per_of:
      ap_pairs = rng.sample(ap_pairs,max_triplets_per_of)

    for a,p in ap_pairs:
      neg_oid = rng.choice([x for x in outfit_ids if x!=oid])
      n = rng.choice(grouped[neg_oid])

      triplets.append((a,p,n))

      if rng.random() <0.5:
        triplets.append((p,a,n))
  rng.shuffle(triplets)

  return pd.DataFrame(triplets,columns=["anchor","positive","negative"])


In [ ]:
cleaned_df= pd.read_parquet("cleaned_parquet.parquet")

triplets = make_triplets(cleaned_df,max_triplets_per_of=50,seed=42)

triplets.to_parquet("triplets.parquet", index=False)

print("triplets generated ",len(triplets))


triplets generated  183542


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,Model
import numpy as np

IMG_SIZE = (224, 224)

# Data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.1),
], name="data_augmentation")

def preprocess_image(path, augment=False):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    if augment:
        img = data_augmentation(img)
    return img

def triplet_generator(df, batch_size=32, training=True):
    anchor_paths = df['anchor'].values
    positive_paths = df['positive'].values
    negative_paths = df['negative'].values

    ds = tf.data.Dataset.from_tensor_slices((anchor_paths, positive_paths, negative_paths))

    def load_triplets(a, p, n):
        x = {
            "anchor": preprocess_image(a, augment=training),   # only augment in training
            "positive": preprocess_image(p, augment=training),
            "negative": preprocess_image(n, augment=False),    # keep negatives clean
        }
        y = tf.zeros((1,), dtype=tf.float32)
        return x, y

    ds = ds.map(load_triplets, num_parallel_calls=tf.data.AUTOTUNE)

    if training:
        ds = ds.shuffle(1024)   # shuffle only in training

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


# ✅ Use smaller batch
train_dataset = triplet_generator(
    triplets.sample(frac=0.8, random_state=42), batch_size=32, training=True
)
val_dataset = triplet_generator(
    triplets.drop(triplets.sample(frac=0.8, random_state=42).index),
    batch_size=32,
    training=False
)


In [ ]:
from tensorflow.keras import layers, Model

def build_embedding_model(embedding_dim=256, trainable_layers=20):
    base = tf.keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",
        pooling="avg",
        input_shape=(224,224,3)
    )
    # Freeze all except last N layers
    for layer in base.layers[:-trainable_layers]:
        layer.trainable = False
    for layer in base.layers[-trainable_layers:]:
        layer.trainable = True

    inputs = tf.keras.Input(shape=(224,224,3))
    x = base(inputs, training=True)
    x = layers.Dense(embedding_dim)(x)
    outputs = layers.Lambda(lambda t: tf.nn.l2_normalize(t, axis=-1))(x)
    return Model(inputs, outputs, name="embedding_model")

embedding_model = build_embedding_model()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
class TripletLoss(tf.keras.losses.Loss):
    def __init__(self, margin=0.3):
        super().__init__()
        self.margin = margin

    def call(self, y_true, y_pred):
        # y_pred shape: (batch, 3*embedding_dim)
        # split into three tensors each (batch, embedding_dim)
        anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)

        # embeddings should be L2-normalized already
        sim_ap = tf.reduce_sum(anchor * positive, axis=-1)
        sim_an = tf.reduce_sum(anchor * negative, axis=-1)

        loss = tf.maximum(0.0, sim_an - sim_ap + self.margin)
        return tf.reduce_mean(loss)


In [ ]:
class TripletAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name="triplet_accuracy", **kwargs):
        super().__init__(name=name, **kwargs)
        self.correct = self.add_weight(name="correct", initializer="zeros")
        self.total = self.add_weight(name="total", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)
        sim_ap = tf.reduce_sum(anchor * positive, axis=-1)
        sim_an = tf.reduce_sum(anchor * negative, axis=-1)
        correct = tf.reduce_sum(tf.cast(sim_ap > sim_an, tf.float32))
        self.correct.assign_add(correct)
        self.total.assign_add(tf.cast(tf.size(sim_ap), tf.float32))

    def result(self):
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        self.correct.assign(0.0)
        self.total.assign(0.0)


class RecallAtK(tf.keras.metrics.Metric):
    def __init__(self, k=5, name="recall_at_k", **kwargs):
        super().__init__(name=name, **kwargs)
        self.k = k
        self.correct = self.add_weight(name="correct", initializer="zeros")
        self.total = self.add_weight(name="total", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # For triplet (anchor, positive, negative) a simple proxy is same as accuracy:
        anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)
        sim_ap = tf.reduce_sum(anchor * positive, axis=-1)
        sim_an = tf.reduce_sum(anchor * negative, axis=-1)
        correct = tf.reduce_sum(tf.cast(sim_ap > sim_an, tf.float32))
        self.correct.assign_add(correct)
        self.total.assign_add(tf.cast(tf.size(sim_ap), tf.float32))

    def result(self):
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        self.correct.assign(0.0)
        self.total.assign(0.0)


In [ ]:
# Cosine LR schedule
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=3e-5,
    first_decay_steps=1000
)

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    weight_decay=1e-5
)

def build_triplet_model(embedding_model):
  anchor_in= tf.keras.Input(shape=(224,224,3),name='anchor')
  positive_in = tf.keras.Input(shape=(224,224,3),name='positive')
  negative_in= tf.keras.Input(shape=(224,224,3),name='negative')

  anchor_emb= embedding_model(anchor_in)
  positive_emb= embedding_model(positive_in)
  negative_emb= embedding_model(negative_in)

  merged = layers.Concatenate(axis=-1)([anchor_emb,positive_emb,negative_emb])
  return Model(inputs=[anchor_in,positive_in,negative_in],outputs=merged)

triplet_model = build_triplet_model(embedding_model)

triplet_model.compile(
    optimizer=optimizer,
    loss=TripletLoss(margin=0.5),
    metrics=[TripletAccuracy(), RecallAtK(k=5)]
)

In [ ]:
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=7, restore_best_weights=True
)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "best_triplet_model.keras", monitor="val_loss",
    save_best_only=True, verbose=1
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7, verbose=1
)

history = triplet_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stop, checkpoint, reduce_lr, tensorboard_cb],
    verbose=1
)

print("✅ Training finished. Best model saved as best_triplet_model.keras")
print(f"Run in Colab to see TensorBoard:\n%tensorboard --logdir {log_dir}")


Epoch 1/30
 392/4589 ━━━━━━━━━━━━━━━━━━━━ 1:05:27 936ms/step - loss: 0.3370 - recall_at_k: 0.7274 - triplet_accuracy: 0.7274

KeyboardInterrupt: 